# Firm Entry and Exit Problem

In [1]:
import numpy as np
import time
import scipy.optimize as opt
from scipy.stats import norm
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

### 1. Define the recursive competitive equilibrium

Household maximise utility:
<br>$\underset{C_{t}, N_{t}, K_{t+1}}{\text{max}} \quad E\sum_{t = 0}^{\infty} \beta^{t} (log(C_{t} + a N_{t})$
<br>$\text{subject to, } C_{t} + (K_{t+1} - (1 - \delta) K_{t}) = w_{t} N_{t} + r_{t} K_{t} , \; \forall t$

Euler equation:
<br>$C_{t} = \beta (C_{t+1})^{-1} ((1 - \delta) + r_{t+1})$

Firms maximize profit:
<br>$\underset{w_{t}, r_{t}}{\text{max}} \quad e^{z_{t}} K_{t}^{\theta} N_{t}^{\nu} - w_{t} N_{t} - K_{t} r_{r}$

The free entry condition:
<br>$c_{e} \leq \int v(\varepsilon, 0) \nu(d \varepsilon)$

Market clearing conditions: 
<br>$N(z, g)=\int n(\varepsilon, k ; z, g) g(\varepsilon, k) d \varepsilon d k$
<br>$C(z, g)=\int y(\varepsilon, k ; z, g)  - i(\varepsilon, k ; z, g) d \varepsilon d k$

* The distribution of active firms $g^*(\epsilon,k)$ is stationary

In [2]:
# Set parameters
theta = 0.21
nu = 0.64
delta = 0.1
beta = 0.96
phi = 0.5
rho = 0.9
sigma = 0.02
n_epsilon = 10
Nbar = 0.6

# Steady state equation
rbar = (1/beta) - (1 - delta)
Kbar = (rbar/(Nbar**(nu)*theta))**(1/(theta - 1))
wbar = nu*Kbar**(theta)*Nbar**(nu - 1)
cbar = wbar*Nbar + rbar*Kbar - Kbar + (1 - delta)*Kbar

def Tauchen(n_epsilon, rho, sigma, m = 3):
    zmax = m*np.sqrt(sigma**(2)/(1 - rho**(2)))
    zmin = - m*np.sqrt(sigma**(2)/(1 - rho**(2)))
    z=[0]*n_epsilon
    ztilde = [0]*(n_epsilon-1)
    for i in range(n_epsilon-1):
        if i == 0:
            z[i] = zmin
        else:
            z[i] = zmin + ((zmax - zmin)/(n_epsilon-1))*(i)
    z[n_epsilon-1] = (zmax)
    for i in range(n_epsilon-1):
        ztilde[i] = (z[i+1] + z[i])/2
    pi = np.zeros([n_epsilon, n_epsilon])
    for i in range(n_epsilon):
        for j in range(n_epsilon):
            if j == 0:
                pi[i][j] = norm.cdf((ztilde[j] - rho*z[i])/sigma)
            elif j in range(1, n_epsilon-1):
                pi[i][j] = norm.cdf((ztilde[j] - rho*z[i])/sigma) - norm.cdf((ztilde[j-1] - rho*z[i])/sigma)
            elif j == (n_epsilon -1):
                pi[i][j] = 1 - norm.cdf((ztilde[j-1] - rho*z[i])/sigma)
    return z, pi

testz, testpi = Tauchen(n_epsilon, rho, sigma)
print(testz)
print(testpi)

[-0.13764944032233709, -0.10706067580626218, -0.07647191129018727, -0.04588314677411236, -0.015294382258037448, 0.015294382258037448, 0.04588314677411237, 0.0764719112901873, 0.10706067580626219, 0.13764944032233709]
[[5.30478170e-01 4.15375002e-01 5.32885772e-02 8.56706028e-04
  1.54439460e-06 2.92857183e-10 5.66213743e-15 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [9.67966302e-02 4.93930466e-01 3.69971746e-01 3.87971726e-02
  5.03259844e-04 7.25892202e-07 1.09655174e-10 1.66533454e-15
  0.00000000e+00 0.00000000e+00]
 [3.71959075e-03 1.21954964e-01 5.23228103e-01 3.23146274e-01
  2.76615366e-02 2.89198006e-04 3.33562232e-07 4.01295663e-11
  4.44089210e-16 0.00000000e+00]
 [2.52812798e-05 5.78316444e-03 1.54270165e-01 5.43701344e-01
  2.76746317e-01 1.93110204e-02 1.62557651e-04 1.49849563e-07
  1.43532963e-11 1.11022302e-16]
 [2.82551739e-08 4.80546862e-05 8.83083400e-03 1.91241547e-01
  5.54230974e-01 2.32360418e-01 1.31987073e-02 8.93711456e-05
  6.58100220e-08 5.01743092e-12